# Ally Financial API Cheat Sheet
* Refer to the documentation for additional routes:
* https://www.ally.com/api/invest/documentation/getting-started/

In [2]:
import requests
from requests_oauthlib import OAuth1
from config import (api_key, secret, oath_token, oath_secret)

import pandas as pd
import sqlalchemy
import numpy as np

import sqlite3
from sqlite3 import Error

import matplotlib.pyplot as plt
import datetime as d
import xmltodict

In [3]:
#authentication 
auth = OAuth1(api_key, secret, oath_token, oath_secret)

# Time and Sales for Stocks Example
* documentation: https://www.ally.com/api/invest/documentation/market-timesales-get/ 
* base url: https://api.tradeking.com/
* route: v1/market/timesales.json
* query: ?symbols=MSFT&startdate=2019-05-03&interval=1min

In [12]:
#url 
url = 'https://api.tradeking.com/v1/market/timesales.json?symbols=MSFT&startdate=2019-05-03&interval=1min'

In [14]:
#api request
response = requests.get(url, auth = auth).json()


In [15]:
#send to data frame and format data types
df = pd.DataFrame(response["response"]["quotes"]["quote"])
df = df.sort_values(['datetime'], ascending = False)
df['date'] = pd.to_datetime(df['date'])
df['datetime'] = pd.to_datetime(df['datetime'],  utc=False).dt.tz_convert('US/Central')
df['hi'] = df["hi"].astype(float)
df['incr_vol'] = df["incr_vl"].astype(float)
df['last'] = df["last"].astype(float)
df['lo'] = df["lo"].astype(float)
df['opn'] = df["opn"].astype(float)
df['vl'] = df['vl'].astype(float)
df.head()

date                  datetime       hi incr_vl     last      lo  \
119 2019-05-03 2019-05-03 10:29:00-05:00  128.580   21196  128.570  128.54   
118 2019-05-03 2019-05-03 10:28:00-05:00  128.575   29916  128.555  128.54   
117 2019-05-03 2019-05-03 10:27:00-05:00  128.590   30959  128.565  128.55   
116 2019-05-03 2019-05-03 10:26:00-05:00  128.610   75718  128.555  128.52   
115 2019-05-03 2019-05-03 10:25:00-05:00  128.640   26497  128.595  128.57   

          opn             timestamp       vl  incr_vol  
119  128.5600  2019-05-03T15:30:41Z  21196.0   21196.0  
118  128.5650  2019-05-03T15:30:41Z  29916.0   29916.0  
117  128.5600  2019-05-03T15:30:41Z  30959.0   30959.0  
116  128.6000  2019-05-03T15:30:41Z  75718.0   75718.0  
115  128.5836  2019-05-03T15:30:41Z  26497.0   26497.0

In [7]:
#resample the time value to be greater than 1 min as needed. Example: 30 min resample for last price
df.set_index(df['datetime'], inplace = True)
df.head()
df_resample30 = df.resample(rule = '30min', label = 'right').last()
df_resample30.head()

date                  datetime       hi  \
datetime                                                                  
2019-05-03 09:00:00-05:00 2019-05-03 2019-05-03 08:59:00-05:00  127.865   
2019-05-03 09:30:00-05:00 2019-05-03 2019-05-03 09:29:00-05:00  128.025   
2019-05-03 10:00:00-05:00 2019-05-03 2019-05-03 09:59:00-05:00  128.580   
2019-05-03 10:30:00-05:00 2019-05-03 2019-05-03 10:24:00-05:00  128.625   

                          incr_vl    last      lo      opn  \
datetime                                                     
2019-05-03 09:00:00-05:00   98772  127.83  127.80  127.820   
2019-05-03 09:30:00-05:00   48653  128.02  127.97  128.000   
2019-05-03 10:00:00-05:00  117898  128.49  128.49  128.545   
2019-05-03 10:30:00-05:00   23806  128.58  128.57  128.615   

                                      timestamp        vl  incr_vol  
datetime                                                             
2019-05-03 09:00:00-05:00  2019-05-03T15:25:47Z   98772.0   98772.0  
2019-05-03 09:30:00-05:00  2019-05-03T15:25:47Z   48653.0   48653.0  
2019-05-03 10:00:00-05:00  2019-05-03T15:25:47Z  117898.0  117898.0  
2019-05-03 10:30:00-05:00  2019-05-03T15:25:47Z   23806.0   23806.0

# Options Search Example
* Provides market depth for options
* Documentation: https://www.ally.com/api/invest/documentation/market-options-search-get-post/
* base url: https://api.tradeking.com/
* route: v1/market/timesales.json
* query: ?symbol=MSFT&query=xyear-eq%3A2019%20AND%20xmonth-eq%3A06%20AND%20strikeprice-eq%3A140


* Query breakdown:
    * exipiration year equals 2019:
    * xyear-eq%3A 2019

    * and:
    * %20AND%20

    * expiration month equals 06:
    * xmonth-eq%3A 06

    * and strike price equals 140:
    * %20AND%20 strikeprice -eq%3A 140


* Operators:
    * lt :	less than
    * gt :	greater than
    * gte :	greater than or equal to
    * lte :	less than or equal to
    * eq :	equal to

In [17]:
url = 'https://api.tradeking.com/v1/market/options/search.json?symbol=MSFT&query=xyear-eq%3A2019%20AND%20xmonth-eq%3A06%20AND%20strikeprice-eq%3A140'
response = requests.get(url, auth = auth).json()

In [18]:
df = pd.DataFrame(response["response"]["quotes"]["quote"])
df


ask ask_time asksz basis    bid bid_time bidsz     chg chg_sign   chg_t  \
0   0.13    11:53   356    na   0.10    11:52    25  0.0500        u  0.0500   
1  14.10    11:53    10    na   9.90    11:53    10  0.0000        e  0.0000   
2   0.21    11:53    12    na   0.17    11:52    84  0.0100        u  0.0100   
3  14.15    11:53     1    na  10.10    11:53    65  0.0000        e  0.0000   
4   0.28    11:53   117    na   0.26    11:53     7  0.0900        u  0.0900   
5  12.10    11:53     4    na  12.00    11:53     4  0.0000        e  0.0000   

   ...   vl  vwap   wk52hi wk52hidate  wk52lo wk52lodate     xdate xday  \
0  ...    4  0.13    1.320   20190425   0.070   20190502  20190607   07   
1  ...    0    na   0.0000   00000000  0.0000   00000000  20190607   07   
2  ...    7  0.19   0.0000   00000000  0.0000   00000000  20190614   14   
3  ...    0    na   0.0000   00000000  0.0000   00000000  20190614   14   
4  ...  314  0.26   1.8300   20181003  0.0200   20190208  20190621   21   
5  ...    0    na  36.5900   20181123  9.8400   20190425  20190621   21   

  xmonth xyear  
0     06  2019  
1     06  2019  
2     06  2019  
3     06  2019  
4     06  2019  
5     06  2019  

[6 rows x 67 columns]

# Extended Quote Example (Option)
* Works for stocks too
* Documentation: https://www.ally.com/api/invest/documentation/market-ext-quotes-get-post/
* base url: https://api.tradeking.com/
* route: v1/market/ext/quotes.json
* query: ?symbols=MSFT190607C00140000

* Option Symbol naming convention:
    * Underlying symbol - MSFT
    * 2 digit expiration year - 19
    * 2 digit expiration month - 06
    * 2 digit expiration day - 07
    * "C" for Call or "P" for Put - C
    * 8 digit strike price - 00140000
    

* Specify desired fields in the query as needed using fids: 
    * i.e. fids=ask,bid,vol

In [21]:
url = 'https://api.tradeking.com/v1/market/ext/quotes.json?symbols=MSFT190607C00140000'
response = requests.get(url, auth = auth).json()

In [22]:
df = pd.DataFrame(response["response"]["quotes"]["quote"], index = [0])
df

adp_100 adp_200 adp_50 adv_21 adv_30 adv_90   ask ask_time asksz basis  ...  \
0      na      na     na     na     na     na  0.14    12:04   633    na  ...   

   vwap wk52hi wk52hidate wk52lo wk52lodate     xdate xday xmonth xyear yield  
0  0.13  1.320   20190425  0.070   20190502  20190607   07     06  2019    na  

[1 rows x 95 columns]

# Market News
* Documentation: https://www.ally.com/api/invest/documentation/market-news-search-get-post/
* base url: https://devapi.invest.ally.com/
* route: v1/market/news/search.json

Example: get_news('spy', 5) 
* Returns 5 latest news articles for SPY

In [ ]:
def get_news(sym, maxhits):
    baseurl = 'https://devapi.invest.ally.com'
    maxhits = str(maxhits)
    stories = []
    url =f'{baseurl}/v1/market/news/search.json?symbols={sym}&maxhits={maxhits}'
    
    response = requests.get(url, auth = auth).json()
    ids = [response['response']['articles']['article'][n]['id'] 
           for n in range(0, len(response['response']['articles']['article']))]
    
    for v in ids:
        url = f'{baseurl}/v1/market/news/{v}.json'
        response = requests.get(url, auth = auth).json()
        stories.append(response['response']['article'])
    return stories



# Create Stock Order
Creates a live buy, sell, or short-sell order. Returns OrderID

* Documentation: https://www.ally.com/api/invest/documentation/trading/
* base url: https://devapi.invest.ally.com/
* route: v1/accounts/{acct}/orders.xml

FIXML Key

* *Sym*: The symbol (ticker) of the security.

* *Typ*: Price Type. Values include Market: 1, Limit: 2, Stop: 3, Stop Limit: 4

* *Side*: The side of the market. Buy to cover orders are attributed as buy orders with Side = "1". Values include Buy: 1, Sell: 2, Sell Short: 5

* *TmInForce*: Time in Force controls the order duration. Values include Day Order: 0, GTC Order: 1, Market on Close:7

* *Px*: The price of the security if needed for the selected Typ. For example, Px would be required for limits (Typ = "2") or stop limits (Typ="4").

* *Acct*: Your account number. This is also required in the URL.

* *Qty*: The quantity of shares or contracts desired.

*Example* stock_order("GE", 2, 1,1, "1.00", 12345678, 1)
* Creates a GTC order to buy 1 share of GE at 1 dollar a share.

In [ ]:
def stock_order(sym, typ, side, tif, price, acct, qty):
    '''
    Creates a live buy, sell, or short-sell order. Returns OrderID
    FIXML KEY
    Typ:  Market: "1" 
          Limit: "2" 
          Stop: "3" 
          Stop Limit: "4"
    
    Side: Buy: "1" 
          Sell: "2" 
          Sell Short: "5" ‐ 
   
    tif:  Day Order: "0" 
          GTC Order: "1" 
          Market on Close: "7" 
        '''
       
    url = f"https://devapi.invest.ally.com/v1/accounts/{acct}/orders.xml"
    xmlns = 'xmlns="http://www.fixprotocol.org/FIXML-5-0-SP2"'
    order = f'TmInForce="{tif}" Typ="{typ}" Side="{side}" Px="{price}" Acct="{acct}"'
    instrmt = f'SecTyp="CS" Sym="{sym}"'
    orderQty = f'Qty="{qty}"'

    payload = f"<FIXML {xmlns}>\r\n <Order {order}>\r\n <Instrmt {instrmt}/>\r\n <OrdQty {orderQty}/>\r\n  </Order> \r\n </FIXML>"
    
    headers = {
      'TKI_OVERRIDE': 'true',
      'Content-Type': 'application/xml',
     }
    response = requests.post(url,auth = auth,  headers=headers, data = payload)
 
    f = xmltodict.parse(response.text.encode('utf8'))['response']['clientorderid']
    #orderids.append(f)
   
    return f 
    

# Cancle Stock Order
Cancles a stock order based on orderID. 

* Documentation: https://www.ally.com/api/invest/documentation/trading/
* base url: https://devapi.invest.ally.com/
* route: v1/accounts/{acct}/orders.xml


*Example* can_stock_order("GE", 2, 1,1, 12345678, 1, 'SVI-123456789') 
*  Creates a GTC order to buy 1 share of GE at 1 dollar a share.

In [ ]:
def can_stock_order(sym, typ, side, tif, acct, qty, orderids):
    '''
    FIXML KEY
    Typ:  Market: "1" 
          Limit: "2" 
          Stop: "3" 
          Stop Limit: "4"
    
    Side: Buy: "1" 
          Sell: "2" 
          Sell Short: "5" ‐ 
   
    tif:  Day Order: "0" 
          GTC Order: "1" 
          Market on Close: "7" 
    '''
           
    url = f"https://devapi.invest.ally.com/v1/accounts/{acct}/orders.xml"

    xmlns = 'xmlns="http://www.fixprotocol.org/FIXML-5-0-SP2"'
    order = f'TmInForce="{tif}" Typ="{typ}" Side="{side}" OrigID="{orderids}" Acct="{acct}"'
    instrmt = f'SecTyp="CS" Sym="{sym}"'
    orderQty = f'Qty="{qty}"'

    payload = f"<FIXML {xmlns}>\r\n <OrdCxlReq {order}>\r\n <Instrmt {instrmt}/>\r\n <OrdQty {orderQty}/>\r\n  </OrdCxlReq> \r\n </FIXML>"

    headers = {
          'TKI_OVERRIDE': 'true',
          'Content-Type': 'application/xml',
         }

    response = requests.post(url,auth = auth,  headers=headers, data = payload)
    f = xmltodict.parse(response.text.encode('utf8'))['response']
    #c_orders.append(f)

    return f